## Sending email using `Python`

### What is SMTP?

- SMTP stands for Simple Mail Transfer Protocol

- used as a protocol for transfer of electronic mail(e-mail).

### Process of sending email

![SMTP-email-transfer-model](https://upload.wikimedia.org/wikipedia/commons/6/69/SMTP-transfer-model.svg)

<em>Image courtsey Wikipedia</em>

#### Using the `smtplib` in `python`

In [6]:
import smtplib


server = smtplib.SMTP('localhost', 1025)

ConnectionRefusedError: [Errno 111] Connection refused

#### Starting up a smtp server using python

```sh
python -m smtpdb -n DebuggingServer -c localhost:1025 
```

#### Now retrying to connect to the server

In [5]:
import smtplib

server = smtplib.SMTP('localhost', 1025)

### Sending plain text emails

In [48]:
import smtplib
from email.message import EmailMessage


with smtplib.SMTP('localhost', 1025) as server:
    msg = EmailMessage()
    msg['To'] = 'receiver@domain'
    msg['From'] = 'sender@domain'
    msg['Subject'] = 'Test email'
    msg.set_content('Hi! this was sent from a script')
    
    server.send_message(msg)
    

#### A real world example using `gmail`

In [55]:
import smtplib
from email.message import EmailMessage


msg = EmailMessage()
msg['to'] = 'receiver@domain'
msg['from'] = sender = 'sender@gmail.com'
msg['subject'] = 'Test sending plain text emails through python'
msg.set_content('Hi! let us hope this works')

with smtplib.SMTP_SSL('smtp.gmail.com', 465) as server:
    server.login(sender, "sender's password")
    server.send_message(msg)

SMTPServerDisconnected: Connection unexpectedly closed

### Sending HTML emails

In [44]:
import smtplib
from email.message import EmailMessage


msg = EmailMessage()
msg['To'] = 'receiver@domain'
msg['From'] = 'sender@domain'
msg['Subject'] = 'Testing HTML email'
msg.add_alternative('''\
<html>
    <p>Please <b>visit</b> the <a href="https://www.hackadda.com">link</a>.</p>
</html>
''', subtype='html'
)

with smtplib.SMTP('localhost', 1025) as server:
    server.send_message(msg)

#### Sending both HTML and text emails

In [18]:
import smtplib
from email.message import EmailMessage


msg = EmailMessage()
msg['To'] = 'receiver@domain'
msg['From'] = 'sender@domain'
msg['Subject'] = 'Testing HTML email'
msg.set_content('Please visit the link https://www.hackadda.com')
msg.add_alternative('''\
<html>
    <p>Please <b>visit</b> the <a href="https://www.hackadda.com">link</a>.</p>
</html>
''', subtype='html'
)

with smtplib.SMTP('localhost', 1025) as server:
    server.send_message(msg)

#### Sending emails with large content

- Read the contents from the file and then set the attributes for the email message object.

#### Parsing headers

Suppose, we have a text file 'email_headers.txt' that contains the content as shown below:

```txt
To: Bar <receiver@domain>
From: Foo <sender@domain>
Subject: Testing parsing headers from text template

Wow! This is exceptional

We can send emails by parsing text from a file.
```

In [ ]:
import smtplib
from email.parser import BytesParser
from email.policy import default


with open('email_headers.txt', 'rb') as fp:
    message = BytesParser(policy=default).parse(fp)

with smtplib.SMTP('localhost', 1025) as server:
    server.send_message(message)

### Sending Attachments

#### Sending images

In [9]:
import imghdr

with open('../images/transaction.png', 'rb') as fp:
    img_data = fp.read()

imghdr.what(None, img_data)

'png'

In [10]:
import smtplib
from email.message import EmailMessage
import imghdr


msg = EmailMessage()
msg['to'] = 'receiver@domain'
msg['from'] = 'sender@domain'
msg['subject'] = 'Test sending attachments through python'
msg.preamble = 'You will not see this in a MIME-aware mail reader.\n'
msg.set_content('Please find the image attached below.')

with open('../images/transaction.png', 'rb') as fp:
    img_data = fp.read()

msg.add_attachment(img_data, maintype='image', subtype=imghdr.what(None, img_data))

with smtplib.SMTP('localhost', 1025) as server:
    server.send_message(msg)
    

#### Sending files

In [42]:
import smtplib, ssl
from email.message import EmailMessage
import os
import mimetypes


PATH = '../images/transaction.png'

sender = os.getenv('EMAIL_SENDER')

msg = EmailMessage()
msg['to'] = 'receiver@domain'
msg['from'] = 'sender@domain'
msg['subject'] = 'Test sending attachments through python'
msg.preamble = 'You will not see this in a MIME-aware mail reader.\n'
msg.set_content('Please find the image attached below.')

with open(PATH, 'rb') as fp:
    img_data = fp.read()

# Guess the content type based on the file's extension.  Encoding
# will be ignored, although we should check for simple things like
# gzip'd or compressed files.
ctype, encoding = mimetypes.guess_type(PATH)
if ctype is None or encoding is not None:
    # No guess could be made, or the file is encoded (compressed), so
    # use a generic bag-of-bits type.
    ctype = 'application/octet-stream'
maintype, subtype = ctype.split('/', 1)
msg.add_attachment(img_data, maintype=maintype, subtype=subtype)

with smtplib.SMTP('localhost', 1025) as server:
    server.send_message(msg)
